In [ ]:
#Instalar si se usa por primera vez
!pip install pandas openpyxl python-Levenshtein
!pip install pandas Jinja2
!pip install reportlab
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install gensim
!python -m spacy download es_core_news_sm #No borrar!!! Si no no jala :(

In [ ]:
import pandas as pd
import re
import pandas as pd
import os
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
import datetime
import pandas as pd
import re
import spacy
import nltk
import csv
import gensim
import numpy as np
from collections import Counter
from fuzzywuzzy import fuzz
import io
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from reportlab.graphics.shapes import Circle, Drawing
from reportlab.graphics.renderPDF import draw
import os
import pandas as pd

In [ ]:
#Cosas en español
nltk.download('wordnet')
nltk.download('omw')
nlp = spacy.load("es_core_news_sm")
#COnectores discursivos
def cargar_conectores(path):
    con_disc = set()
    with open(path, newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            con_disc.update(row)
    return con_disc

con_disc = cargar_conectores("/content/drive/MyDrive/conectores dfiscursivos1.csv")
#Más cosas en español
nltk.download('stopwords')
stopword_es = nltk.corpus.stopwords.words('spanish')

In [ ]:
def limpiar(texto):
    if pd.isna(texto):
        return ""

    texto = texto.lower()
    texto = re.sub(r"(100)\b\d+", "", texto)
    texto = re.sub(r"(-1)\b\d+", "", texto)
    texto = re.sub(r"[^\w\s/*-]", "", texto)
    texto = re.sub(r"\s+", " ", texto).strip()
    doc = nlp(texto)
    tokens = [
        token.text for token in doc
        if not token.is_stop
        and token.lemma_ not in con_disc
        and token.text not in stopword_es
    ]

    return " ".join(tokens)

def limpiar_tv(texto):
    texto = texto.lower()
    texto = texto.strip()
    matches = re.findall(r"\(([^()]*)\)", texto)
    if matches:
        for match in matches:
            if '/' in match:
                return match
    return texto
def cociente(text):
    match = re.search(r"(.+?)\s*/\s*(.+)", text)
    if match:
        num = match.group(1).strip()
        den = match.group(2).strip()
        similarity_ratio = fuzz.ratio(num, den)
        return similarity_ratio >= 50
    return False

def comparacion(metodo_p, tipo_r):
    if tipo_r == '(A/B)*100':
        pattern = r".*\s*[xX\*]?\s*100"
        if (re.search(pattern, metodo_p) or re.search(r".*\/.*\\b(programado|planificado|programada|planificada|multiplicado por 100|x100|x 100|X100|X 100|\*100|\* 100)\\b", metodo_p)) and not re.search(r".*\/.*\\b(solicitadas|recibidas|requeridas|demandadas|gestionadas|tramitadas)\\b", metodo_p):
            return 'Sí'
    elif tipo_r == 'Tasa de variación':
        pattern = r'\s*1(?:\s*[xX\*]?\s*100)?'
        if cociente(limpiar_tv(metodo_p)) and (re.search(pattern, metodo_p) or re.search(r".*\/.*\\b(anterior|pasado|año pasado|mes pasado|año anterior|ciclo anterior|ciclo pasado|año t-1|año t - 1| año t- 1|año t -1|t-1|-1*100|-1 * 100|-1 x 100| -1 X 100|t - 1|t- 1|t -1|n-1|n - 1|n -1|n- 1|t1|t 1)\\b",metodo_p)):
                return 'Sí'
    elif tipo_r == 'Absoluto':
        pattern = r".*\s*/\s*.*"
        if not re.search(pattern, metodo_p):
                return 'Sí'
    return 'No'
#Método para comparar los componentes
def comp_met(comp_1, comp_2):
  similitud = fuzz.ratio(comp_1, comp_2)
  if similitud >= 50:
    return "Sí"
  else:
    return "No"


def comparar_otros(row):
    if row['TIPO_RELATIVO_METODO_COMPARACION'] == 'No' or row['TIPO_RELATIVO_METODO_COMPARACION'] == 'Otro':
        metodo_p = row['METODO_PROCESADO']
        if (re.search( r".*\s*[xX\*]?\s*100", metodo_p) or re.search(r".*\/.*\\b(programado|planificado|programada|planificada|multiplicado por 100|x100|x 100|X100|X 100|\*100|\* 100)\\b", metodo_p))  and not (re.search(r".*\/.*\b(solicitadas|recibidas|requeridas|demandadas|gestionadas|tramitadas)\b", metodo_p) and re.search(r'-\s*1.*?[xX\*]?\s*100', metodo_p)):
            return '(A/B)*100'
        elif cociente(metodo_p) and (re.search(r'\s*1.*?[xX\*]?\s*100', metodo_p) or re.search(r".*\/.*\\b(anterior|pasado|año pasado|mes pasado|año anterior|ciclo anterior|ciclo pasado|año t-1|año t - 1| año t- 1|año t -1||t-1|t - 1|t- 1|t -1|n-1|n - 1|n -1|n- 1|t1|t 1)\\b",metodo_p)):
            return 'Tasa de variación'
        elif not (re.search(r".*\s*/\s*.*" , metodo_p) or (p in metodo_p for p in ['cociente', '/', 'entre', 'dividido', 'respecto', 'dividiendo'])):
            return 'Absoluto'
    return 'No'

def comparacion_indicador(info):
    columnas = ["TIPO_RELATIVO", "METODO_CALCULO_INDICADOR", "NOMBRE_INDICADOR", "DEFINICION_INDICADOR"]
    if not all(col in info.columns for col in columnas):
        raise ValueError(f"Faltan las siguientes columnas: {', '.join(columnas)}")
    info_copy = info.copy()

    #Comparación entre el método de cálculo y el tipo relativo
    info_copy['METODO_PROCESADO'] = info_copy.apply(
        lambda row: row['METODO_CALCULO_INDICADOR'] if pd.isna(row['TIPO_RELATIVO']) or row['TIPO_RELATIVO'] == "Tasa de variación" else limpiar(row['METODO_CALCULO_INDICADOR']),
        axis=1
    )
    info_copy['TIPO_RELATIVO_METODO_COMPARACION'] = info_copy.apply(
        lambda row: comparacion(row['METODO_PROCESADO'], row['TIPO_RELATIVO']),
        axis=1
    )

    #Limpiar las demás columnas para evaluar
    info_copy['NOMBRE_PROCESADO'] = info_copy.apply(
        lambda row: limpiar(row['NOMBRE_INDICADOR']),
        axis=1
    )
    info_copy['DEFINICION_PROCESADO'] = info_copy.apply(
        lambda row: limpiar(row['DEFINICION_INDICADOR']),
        axis=1
    )

    info_copy['OTRA_COINCIDENCIA'] = None

    #COMPARACIÓN
    otra_coin = (info_copy['TIPO_RELATIVO_METODO_COMPARACION'] == 'No') & (~info_copy['TIPO_RELATIVO_METODO_COMPARACION'].isna())
    info_copy.loc[otra_coin, 'OTRA_COINCIDENCIA'] = info_copy.loc[otra_coin].apply(comparar_otros, axis=1)

    otra_coin = (info_copy['TIPO_RELATIVO'] == 'Otro')
    info_copy.loc[otra_coin, 'OTRA_COINCIDENCIA'] = info_copy.loc[otra_coin].apply(comparar_otros, axis=1)

    #Comparación nombre y definición
    info_copy['NOMBRE_DEFINICION_COMPARACION'] = info_copy.apply(
        lambda row: comp_met(str(row['NOMBRE_PROCESADO']), str(row['DEFINICION_PROCESADO'])),
        axis=1
    )

    #Comparación definición y método de cálculo
    info_copy['DEFINICION_METODO_COMPARACION'] = info_copy.apply(
        lambda row: comp_met(str(row['DEFINICION_PROCESADO']), str(row['METODO_PROCESADO'])),
        axis=1
    )

    return info_copy[['METODO_CALCULO_INDICADOR', 'TIPO_RELATIVO_METODO_COMPARACION', 'OTRA_COINCIDENCIA', 'NOMBRE_DEFINICION_COMPARACION', 'DEFINICION_METODO_COMPARACION']]